In [3]:
import os
import sys
sys.path.append(os.path.abspath("/ocean/projects/asc170022p/shg121/PhD/Project_Pruning"))
import h5py
import pickle
from sklearn.utils import shuffle
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import time
import pickle

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import utils
from model_factory.models import Classifier
from model_factory.model_meta import Model_Meta
from dataset.dataset_utils import get_dataset_with_attributes, get_transforms
from dataset.dataset_attributes_mnist import Dataset_attributes_mnist

import matplotlib.pyplot as plt

In [4]:
seed = 0
device = utils.get_device()
print(f"Device: {device}")
data_root = "/ocean/projects/asc170022p/shg121/PhD/Project_Pruning/data/MNIST_EVEN_ODD"
json_root = "/ocean/projects/asc170022p/shg121/PhD/Project_Pruning/scripts_data"
model_arch = "Resnet_18"
dataset_name = "CUB200"
pretrained = True
transfer_learning = False
chk_pt_path = "seq_epoch_20.pth.tar"
num_classes = 1
logs = "/ocean/projects/asc170022p/shg121/PhD/Project_Pruning/output"
bb_layer = "layer3"  # layer3
concept_names = ["Zero", "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine"]
img_size = 224
batch_size = 3
epochs = 50
num_workers = 4
class_list = [0, 1]
num_labels = len(class_list)
cav_vector_file = "max_pooled_train_cavs.pkl"
kernel_size={
    "layer3": 14,
    "layer4": 7
}

prune_type = "lt"
lr = 1e-3
ITERATION  = 35
prune_percent = 10
start_iter = 0
end_iter = 100
resample = False
reinit = True if prune_type=="reinit" else False

data_augmentation=True

Device: cuda


In [ ]:
def get_transform(dataset, size, data_augmentation=False):
    resize = int(size * 0.9)
    if dataset == "CUB200":
        if data_augmentation:
            return transforms.Compose([
                transforms.Resize(size=resize),
                transforms.CenterCrop(size=size),
                transforms.ColorJitter(brightness=0.4, contrast=0.4, hue=0.4, saturation=0.4),
                transforms.RandomRotation(0.4),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            return transforms.Compose([
                transforms.Resize(size=resize),
                transforms.CenterCrop(size=size),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

In [ ]:
train_transform = get_transform(dataset=dataset_name, size=img_size, data_augmentation=data_augmentation)
test_transform = get_transform(dataset=dataset_name, size=img_size, data_augmentation=False)

In [ ]:
path = "/ocean/projects/asc170022p/shg121/PhD/Project_Pruning/data/CUB_200_2011"
file_name = "attributes.npy"

x = np.load(os.path.join(path, file_name))
x.shape

In [ ]:
x[0]

In [ ]:
file_name1 = "original_attributes.npy"
attributes = np.load(os.path.join(path, file_name))
print(attributes.shape)
print(attributes)

In [ ]:
np.sum(attributes, axis=0) 

In [ ]:
attributes[:, np.sum(attributes, axis=0) > 0]

In [ ]:
def clean_names(names: list) -> list:
    names = \
        [
            name.replace("::", "_")
                .replace(":", "_")
                .replace(".", "_")
                .replace(" ", "_")
                .replace("\n", "")
                .replace("-", "")
                .replace("(", "")
                .replace(")", "")
            for name in names
        ]
    return names

In [ ]:
import json
with open(os.path.join(path, "attributes_names.txt"), "r") as f:
     attribute_names : list = json.load(f)

In [ ]:
attribute_names

In [ ]:
attribute_names = np.asarray(clean_names(attribute_names))
print(attribute_names.shape)
print(attribute_names)

In [ ]:
from torchvision.datasets import ImageFolder
def get_dataset(
        data_root,
        json_root,
        dataset_name,
        mode,
        attribute_file
):
    data_json = os.path.join(
        json_root,
        f"{mode}_samples_{dataset_name}.json"
    )

    if os.path.isfile(data_json):
        with open(os.path.join(data_json), "r") as f:
            json_file = json.load(f)
            data_samples = json_file["samples"]

    print(f"Length of the [{mode}] dataset: {len(data_samples)}")
    img_set = ImageFolder(data_root)
    img_dataset = [img_set[index] for index in data_samples]
    attributes = np.load(os.path.join(data_root, attribute_file))[data_samples]
    
    return img_dataset, attributes

In [ ]:
train_set, attributes = get_dataset(
        data_root="/ocean/projects/asc170022p/shg121/PhD/Project_Pruning/data/CUB_200_2011",
        json_root="/ocean/projects/asc170022p/shg121/PhD/Project_Pruning/scripts_data",
        dataset_name="cub",
        mode="val",
        attribute_file="attributes.npy"
)

In [ ]:
class Dataset_attributes_mnist(Dataset):
    def __init__(self, dataset, attributes, transform=None, show_image=False):
        self.dataset = dataset
        self.show_image = show_image
        self.transform = transform
        self.attributes = attributes

    def __getitem__(self, item):
        image = self.dataset[item][0]
        label = self.dataset[item][1]
        attributes = self.attributes[item]
        if self.transform:
            image = self.transform(image)
        return image, label, attributes

    def __len__(self):
        return len(self.dataset)

In [ ]:
import os 
folder = {}
rootdir = "/ocean/projects/asc170022p/shg121/PhD/Project_Pruning/data/CUB_200_2011"
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        key = d.split(".")[0]
        folder[int(key.split("/")[-1])] = d.split(".")[1]
        
folder

In [ ]:
dictionary_items = folder.items()
sorted_items = sorted(dictionary_items)
sorted_items

In [ ]:
labels = []
for x in sorted_items:
    labels.append(x[1])
    
print(labels)

In [ ]:
dataset = Dataset_attributes_mnist(train_set, attributes, test_transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
print(f"Train set size: {len(dataloader)}")
data = iter(dataloader)

image, label, attribute = next(data)
print(image.size())
print(label.item())
print(attribute)

fig, ax = plt.subplots(1, 1, figsize=(16, 16))
ax.imshow(np.transpose(image.numpy()[0], (1, 2, 0)), cmap='gray')
ax.text(
        10,
        0,
        f"Label: {labels[label], label}",
        style='normal',
        size='xx-large'
)
plt.show()

In [6]:
import lth_pruning.pruning_utils as pruning_utils
import copy

logs = "/ocean/projects/asc170022p/shg121/PhD/Project_Pruning/Results"
num_classes = 200
pretrained = True
transfer_learning = False
model_arch = "Resnet_50"
dataset_name = "cub"

checkpoint_path = os.path.join(logs, "chk_pt", "Pruning", model_arch, dataset_name)
utils.create_dir(
        path_dict={
            "path_name": checkpoint_path,
            "path_type": "checkpoint"
})

model = Classifier("Resnet_50", n_classes=200, dataset_name="cub", pretrained=True).to(device)
for name, param in model.named_parameters():
    print(name, param[0])
    break

print(model)
bb_initial_state_dict = copy.deepcopy(model.state_dict())
torch.save(
    model,
    os.path.join(checkpoint_path, f"initial_state_dict_prune_type_{prune_type}.pth.tar")
)

checkpoint directory is created successfully at:
/ocean/projects/asc170022p/shg121/PhD/Project_Pruning/Results/chk_pt/Pruning/Resnet_50/cub
model.conv1.weight tensor([[[ 0.0133,  0.0147, -0.0154, -0.0230, -0.0409, -0.0430, -0.0708],
         [ 0.0041,  0.0058,  0.0149,  0.0206,  0.0022, -0.0209, -0.0385],
         [ 0.0223,  0.0236,  0.0161,  0.0588,  0.1028,  0.0626,  0.0520],
         [ 0.0232,  0.0042, -0.0459, -0.0487, -0.0164,  0.0402,  0.0658],
         [-0.0009,  0.0278, -0.0101, -0.0554, -0.1272, -0.0766,  0.0078],
         [ 0.0036,  0.0480,  0.0621,  0.0844,  0.0243, -0.0337, -0.0157],
         [-0.0800, -0.0322, -0.0178,  0.0342,  0.0354,  0.0224,  0.0017]],

        [[-0.0185,  0.0114,  0.0239,  0.0551,  0.0537,  0.0440, -0.0095],
         [-0.0077,  0.0189,  0.0680,  0.1305,  0.1596,  0.1461,  0.1200],
         [-0.0460, -0.0761, -0.0896, -0.0033,  0.1211,  0.1670,  0.1762],
         [-0.0386, -0.1214, -0.2471, -0.3167, -0.2521, -0.0763,  0.0408],
         [ 0.0288,  0.013

In [ ]:
cls = torch.load(os.path.join(checkpoint_path, f"initial_state_dict_prune_type_{prune_type}.pth.tar"))
for name, param in cls.named_parameters():
    print(name, param[0])
    break

In [ ]:
print(cls)
y = cls(image.to(device))
print(y)
print(y.size())
prob = torch.nn.Softmax(dim=1)
prob_tensor = prob(y)
print(prob_tensor)
print(prob_tensor.size())
print(torch.sum(prob_tensor))

In [ ]:
bb_initial_state_dict_1 = copy.deepcopy(cls.state_dict())

In [ ]:
for name, param in cls.named_parameters():
    print(name, param[0])
    break
print(bb_initial_state_dict_1)

In [ ]:
import torchvision
m = torchvision.models.resnet18()
print(m)

In [ ]:
with tqdm(total=len(dataloader)) as t:
    for data, target, _ in dataloader:
        print(data.size())

In [9]:
model_path = "/ocean/projects/asc170022p/shg121/PhD/Project_Pruning/Results/chk_pt/Pruning/Resnet_50/cub/best_val_prune_iteration_0_model_lt.pth.tar"
x = torch.load(model_path)
print(x)

OrderedDict([('model.conv1.weight', tensor([[[[ 1.0860e-02,  1.3710e-02, -1.5212e-02,  ..., -4.0519e-02,
           -4.2640e-02, -7.0131e-02],
          [-3.3791e-04,  4.2600e-03,  1.6310e-02,  ..., -8.5077e-04,
           -2.0724e-02, -3.8172e-02],
          [ 1.9243e-02,  2.1147e-02,  1.6683e-02,  ...,  1.0387e-01,
            6.2418e-02,  5.4913e-02],
          ...,
          [-8.9521e-04,  2.3875e-02, -1.0011e-02,  ..., -1.2604e-01,
           -7.5915e-02,  7.6988e-03],
          [-4.9698e-04,  4.4813e-02,  5.9035e-02,  ...,  2.3007e-02,
           -3.3351e-02, -1.5566e-02],
          [-7.9274e-02, -3.1927e-02, -1.7640e-02,  ...,  3.1900e-02,
            1.7295e-02, -9.0343e-04]],

         [[-1.8280e-02,  9.8722e-03,  2.4775e-02,  ...,  5.6540e-02,
            4.4624e-02, -9.3813e-03],
          [-7.6550e-03,  1.3795e-02,  6.5548e-02,  ...,  1.5924e-01,
            1.4484e-01,  1.2192e-01],
          [-4.5602e-02, -7.5385e-02, -8.8827e-02,  ...,  1.1863e-01,
            1.6328e-01

In [14]:
print(x["model.fc.weight"].size())
print(x["model.fc.bias"].size())

torch.Size([200, 2048])
torch.Size([200])


In [13]:
for name, param in model.named_parameters():
    print(name, param.size())

model.conv1.weight torch.Size([64, 3, 7, 7])
model.bn1.weight torch.Size([64])
model.bn1.bias torch.Size([64])
model.layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
model.layer1.0.bn1.weight torch.Size([64])
model.layer1.0.bn1.bias torch.Size([64])
model.layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
model.layer1.0.bn2.weight torch.Size([64])
model.layer1.0.bn2.bias torch.Size([64])
model.layer1.0.conv3.weight torch.Size([256, 64, 1, 1])
model.layer1.0.bn3.weight torch.Size([256])
model.layer1.0.bn3.bias torch.Size([256])
model.layer1.0.downsample.0.weight torch.Size([256, 64, 1, 1])
model.layer1.0.downsample.1.weight torch.Size([256])
model.layer1.0.downsample.1.bias torch.Size([256])
model.layer1.1.conv1.weight torch.Size([64, 256, 1, 1])
model.layer1.1.bn1.weight torch.Size([64])
model.layer1.1.bn1.bias torch.Size([64])
model.layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
model.layer1.1.bn2.weight torch.Size([64])
model.layer1.1.bn2.bias torch.Size([64])
model.layer1.1.conv3.we

In [15]:
model.load_state_dict(x)

<All keys matched successfully>

In [17]:
n_gpu = torch.cuda.device_count()
n_gpu

1